<a href="https://colab.research.google.com/github/ruelanthonyb/sales-predictions/blob/main/Sales_Predictions_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
filename = '/content/sales_predictions_2023.csv'

df_sales = pd.read_csv(filename)
df_sales.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


1. How many rows and columns?

In [ ]:
df_sales.shape

(8523, 12)

In [ ]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


2. What are the datatypes of each variable?

In [ ]:
df_sales.dtypes

Item_Identifier               object
Item_Weight                  float64
Item_Fat_Content              object
Item_Visibility              float64
Item_Type                     object
Item_MRP                     float64
Outlet_Identifier             object
Outlet_Establishment_Year      int64
Outlet_Size                   object
Outlet_Location_Type          object
Outlet_Type                   object
Item_Outlet_Sales            float64
dtype: object

3. Are there duplicates? If so, drop any duplicates.

In [ ]:
duplicate_rows = df_sales[df_sales.duplicated()]

# No rows are duplicated entirely across all columns
print(duplicate_rows)

Empty DataFrame
Columns: [Item_Identifier, Item_Weight, Item_Fat_Content, Item_Visibility, Item_Type, Item_MRP, Outlet_Identifier, Outlet_Establishment_Year, Outlet_Size, Outlet_Location_Type, Outlet_Type, Item_Outlet_Sales]
Index: []


4. Identify missing values.

In [ ]:
df_sales.isna().sum()

Item_Identifier                 0
Item_Weight                  1463
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64

In [ ]:
# Total missing values
df_sales.isna().sum().sum()

3873

In [ ]:
rows_with_missing_values = df_sales[df_sales.isna().any(axis=1)]
print(rows_with_missing_values)

     Item_Identifier  Item_Weight Item_Fat_Content  Item_Visibility  \
3              FDX07        19.20          Regular         0.000000   
7              FDP10          NaN          Low Fat         0.127470   
8              FDH17        16.20          Regular         0.016687   
9              FDU28        19.20          Regular         0.094450   
18             DRI11          NaN          Low Fat         0.034238   
...              ...          ...              ...              ...   
8504           NCN18          NaN          Low Fat         0.124111   
8508           FDW31        11.35          Regular         0.043246   
8509           FDG45         8.10          Low Fat         0.214306   
8514           FDA01        15.00          Regular         0.054489   
8519           FDS36         8.38          Regular         0.046982   

                  Item_Type  Item_MRP Outlet_Identifier  \
3     Fruits and Vegetables  182.0950            OUT010   
7               Snack Foods  

5. Decide on how to address the missing values and do it! (This requires your judgement, so explain your choice).

In [ ]:
# The code fills the missing values in the Item_Weight column of the DataFrame df_sales with the average weight of products
# that have the same Item_Type and Item_Fat_Content. This approach makes the assumption that items of a particular type
# and fat content will likely have similar weights.

df_sales['Item_Weight'] = df_sales.groupby(['Item_Type', 'Item_Fat_Content'])['Item_Weight'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
# The code first determines the most common (modal) Outlet_Size for each Outlet_Type.
# It then fills the missing values in the Outlet_Size column based on the type of outlet.
# This approach assumes that outlets of a particular type likely have a common size.

mode_for_outlet_type = df_sales.groupby('Outlet_Type').Outlet_Size.apply(lambda x: x.mode().iloc[0]).to_dict()

df_sales['Outlet_Size'] = df_sales['Outlet_Size'].fillna(df_sales['Outlet_Type'].map(mode_for_outlet_type))

6. Confirm that there are no missing values after addressing them.

In [ ]:
df_sales.isna().sum()

Item_Identifier              0
Item_Weight                  0
Item_Fat_Content             0
Item_Visibility              0
Item_Type                    0
Item_MRP                     0
Outlet_Identifier            0
Outlet_Establishment_Year    0
Outlet_Size                  0
Outlet_Location_Type         0
Outlet_Type                  0
Item_Outlet_Sales            0
dtype: int64

7. Find and fix any inconsistent categories of data (example: fix cat, Cat, and cats so that they are consistent)

In [ ]:
# Showing inconsistent categories of 'Item_Fat_Content' column
print(df_sales['Item_Fat_Content'].unique())

['Low Fat' 'Regular' 'low fat' 'LF' 'reg']


In [ ]:
# fixing inconsistencies by creating a mapping to standardize values
fat_content_mapping = {
    'Low Fat': 'Low Fat',
    'low fat': 'Low Fat',
    'LF': 'Low Fat',
    'Regular': 'Regular',
    'reg': 'Regular'
}

df_sales['Item_Fat_Content'] = df_sales['Item_Fat_Content'].map(fat_content_mapping).fillna(df_sales['Item_Fat_Content'])

In [ ]:
# validating the changes made
print(df_sales['Item_Fat_Content'].unique())

['Low Fat' 'Regular']


8. For any numerical columns, obtain the summary statistics of each (min, max, mean)

In [ ]:
summary_statistics = df_sales.describe()

min_values = summary_statistics.loc['min']
max_values = summary_statistics.loc['max']
mean_values = summary_statistics.loc['mean']

print("Minimum Values:\n", min_values, "\n")
print("Maximum Values:\n", max_values, "\n")
print("Mean Values:\n", mean_values)

Minimum Values:
 Item_Weight                     4.555
Item_Visibility                 0.000
Item_MRP                       31.290
Outlet_Establishment_Year    1985.000
Item_Outlet_Sales              33.290
Name: min, dtype: float64 

Maximum Values:
 Item_Weight                     21.350000
Item_Visibility                  0.328391
Item_MRP                       266.888400
Outlet_Establishment_Year     2009.000000
Item_Outlet_Sales            13086.964800
Name: max, dtype: float64 

Mean Values:
 Item_Weight                    12.859428
Item_Visibility                 0.066132
Item_MRP                      140.992782
Outlet_Establishment_Year    1997.831867
Item_Outlet_Sales            2181.288914
Name: mean, dtype: float64
